In [133]:
import numpy as np
import itertools

### Read in the data

In [134]:
def read_in(file_path):
    X = []
    y = []
    with open(file_path, 'r') as f:
        not_missing = []
        idx = 0
        for line in f:
            missing = False
            info = line.strip('\n').split(',')
            # label is the first column
            if info[0] == 'republican':
                y.append(1)
            else:
                y.append(0)
            feature = []
            for i in range(1, len(info)):
                if info[i] == 'y':
                    feature.append(1)
                elif info[i] == 'n':
                    feature.append(0)
                else:
                    feature.append(-99)
                    missing = True
            X.append(feature)
            if not missing:
                not_missing.append(idx)
            idx += 1
    X = np.array(X)
    y = np.array(y)
    # test using 0 and 1 in this assignment
    # y[y=='republican'] = 1
    # y[y=='democrat'] = 0
    return X, y, np.array(not_missing)

In [135]:
X, y, not_missing = read_in('hw5_data/congress.data')

In [136]:
data = np.zeros((X.shape[0], X.shape[1]+1), dtype=np.int64)
data[:,0] = y
data[:,1:] = X

In [137]:
# full_data consists of samples with no missing variable.
full_data = data[not_missing]

### Mutual Information

In [138]:
def mutual_info(X, i, j):
    """
    Compute the mutual information between variable i and j in dataset X. 
    """
    values_i = set(X[:,i])
    values_j = set(X[:,j])
    info = 0
    N = X.shape[0] # number of samples
    for val_i in values_i:
        for val_j in values_j:
            idx_i = X[:,i] == val_i
            idx_j = X[:,j] == val_j
            idx_ij = idx_i & idx_j
            info += np.sum(idx_ij)/N * np.log((np.sum(idx_ij)/N) / ((np.sum(idx_i)/N)*(np.sum(idx_j)/N)))
            # print(info)
    return info

In [139]:
i = 1
j = 2
a = full_data[:,1] == 0
b = full_data[:,2] == 0

In [140]:
mutual_info(full_data,1,2)

0.0021366407572655285

In [141]:
mutual_info(full_data,2,1)

0.0021366407572655285

### Chow-Liu Tree

In [142]:
num_vars = full_data.shape[1]
graph_matrix = np.zeros((num_vars, num_vars))
for i in range(num_vars):
    for j in range(num_vars):
        if i != j:
            info = mutual_info(full_data, i, j)
            graph_matrix[i,j] = -info
            graph_matrix[j,i] = -info

In [143]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [144]:
tree = minimum_spanning_tree(graph_matrix).toarray()

In [145]:
for i in range(num_vars):
    for j in range(num_vars):
        if tree[i,j] != 0:
            print(i,j)

0 11
1 12
2 10
3 8
4 0
4 5
5 6
5 8
5 12
5 15
7 16
8 7
9 5
13 2
13 5
14 4


### Expectation Maximization

In [146]:
# In E-step, we need to compute a probability distribution for each missing variable

In [147]:
# First, I need to initialize the parameters of the model

In [148]:
D = data.shape[1] # number of features
N = data.shape[0] # number of samples
MISS = -99
parents = {0}

In [149]:
# missing probabilities
b = np.zeros(data.shape[1])
for i in range(D):
    b[i] = np.sum(data[:,i]==MISS) / N 

In [150]:
# model probabilities
parent_dict = {0: None, 4: 0, 11: 0, 14: 4, 5: 4, 15: 5, 6: 5, 8: 5, 12: 5,
            9: 5, 13: 5, 7: 8, 3: 8, 1: 12, 2: 13, 16: 7, 10: 2}
child_dict = {0: {4,11}, 4: {5,14}, 5: {15,6,8,12,9,13}, 8: {7,3}, 7: {16},
            13: {2}, 2: {10}, 12: {1}}

In [151]:
# theta denotes the parameters of the model, i.e. the conditional probabilities
# initialize them with a gaussian random 
# P(x=1|y=1) = theta[x][1]
# note we save only P(x=1|y), P(x=0|y) = 1 - P(x=1|y)
theta = {}
for child in parent_dict:
    theta[child] = {}
    for val_parent in range(2):
        theta[child][val_parent] = np.random.random()
theta[0] = np.random.random()

In [152]:
theta

{0: 0.8694736043573745,
 1: {0: 0.7755673551919341, 1: 0.49392062731297326},
 2: {0: 0.6249197323851591, 1: 0.385232681467243},
 3: {0: 0.8901420108884716, 1: 0.9504120780364718},
 4: {0: 0.36386473845098544, 1: 0.4921528718066369},
 5: {0: 0.3699313809381112, 1: 0.24314168114980184},
 6: {0: 0.2549628440634404, 1: 0.4362803518136984},
 7: {0: 0.7011872779668703, 1: 0.9128217478203641},
 8: {0: 0.8318112958235429, 1: 0.6424653052580166},
 9: {0: 0.7743635360396006, 1: 0.3477978093601367},
 10: {0: 0.245384477940079, 1: 0.9572560356679367},
 11: {0: 0.7511215419546142, 1: 0.5044032281413467},
 12: {0: 0.7167482586280501, 1: 0.7073072392861205},
 13: {0: 0.6706400187985302, 1: 0.6039886633925833},
 14: {0: 0.12450283073234947, 1: 0.8170253811807481},
 15: {0: 0.8091667251633472, 1: 0.23243568670733927},
 16: {0: 0.5097811011103777, 1: 0.8525328133843914}}

In [153]:
def gen_missing_var_dist(sample, theta):
    missing_vars = np.where(sample==MISS)[0]
    possible_assigns = list(itertools.product(*[range(2) for i in range(len(missing_vars))]))
    prob = prob_non_missing(sample, missing_vars, theta)
    q_dist = {var: {} for var in missing_vars} 
    for var in q_dist:
        q_dist[var] = {i: 0 for i in range(2)}

    for each_assign in possible_assigns:
        current_sample = np.array(sample)
        current_sample[missing_vars] = each_assign
        #print(current_sample)
        # calculate the probability
        current_prob = prob
        for var in missing_vars:
            # add probability P(var|parent(var))
            parent_val = current_sample[parent_dict[var]]
            if current_sample[var] == 1: 
                current_prob *= theta[var][parent_val]
            else:
                current_prob *= 1 - theta[var][parent_val]
            # add probability P(child(var)|var)
            if var in child_dict:
                var_val = current_sample[var]
                for each_child in child_dict[var]:
                    child_val = current_sample[each_child]
                    if child_val == 1:
                        current_prob *= theta[each_child][var_val]
                    else:
                        current_prob *= 1 - theta[each_child][var_val]
        # update the q distribution
        for i, var in enumerate(missing_vars):
            q_dist[var][each_assign[i]] += current_prob

    # normalize the q_dist
    for var in q_dist:
        total = q_dist[var][0] + q_dist[var][1]
        q_dist[var][0] /= total
        q_dist[var][1] /= total
    return q_dist

In [154]:
def prob_non_missing(sample, missing_vars, theta):
    """
    Compute the probability for non missing variables
    """
    prob = theta[0]
    for var in range(1,D):
        parent_var = parent_dict[var]
        if var not in missing_vars and parent_var not in missing_vars:
            parent_val = sample[parent_dict[var]]
            if sample[var] == 1:         
                prob *= theta[var][parent_val]
            else:
                prob *= (1 - theta[var][parent_val])
    return prob

In [155]:
missing_vars = np.where(data[2]==MISS)[0]
prob_non_missing(data[2], missing_vars)

TypeError: prob_non_missing() missing 1 required positional argument: 'theta'

In [156]:
gen_missing_var_dist(data[2])

TypeError: gen_missing_var_dist() missing 1 required positional argument: 'theta'

In [77]:
missing_distribution = []
for i in range(N):
    missing_distribution.append(gen_missing_var_dist(data[i]))

In [78]:
len(missing_distribution)

435

### Maximization Step

In [79]:
# update theta
theta

{0: 0.2909177064627031,
 1: {0: 0.10592523590107272, 1: 0.5270469684350967},
 2: {0: 0.9837656218763154, 1: 0.8952317407403592},
 3: {0: 0.48391260053617224, 1: 0.4336635604735033},
 4: {0: 0.5578871284767573, 1: 0.6062198928638358},
 5: {0: 0.46369778293658026, 1: 0.2652465735126366},
 6: {0: 0.8046543066195474, 1: 0.7201388853664181},
 7: {0: 0.2134570087249894, 1: 0.9154527546180836},
 8: {0: 0.2701472246590333, 1: 0.16472059052856136},
 9: {0: 0.7794901943958975, 1: 0.11875560264523766},
 10: {0: 0.32612221863362656, 1: 0.0015348474408563018},
 11: {0: 0.816703663366438, 1: 0.7068698688730279},
 12: {0: 0.8577608722186013, 1: 0.7390014008439103},
 13: {0: 0.4175192672257879, 1: 0.5499867175513667},
 14: {0: 0.4016183428390353, 1: 0.752276594011051},
 15: {0: 0.39650665949871344, 1: 0.7140247815923975},
 16: {0: 0.8758973881592609, 1: 0.8875794085781239}}

In [89]:
# try to update theta of variable 2 for example
theta_up = {}
for child in parent_dict:
    theta_up[child] = {}
    for val_parent in range(2):
        theta_up[child][(0,val_parent)] = 0
        theta_up[child][(1,val_parent)] = 0

theta_up[0] = {}
theta_up[0][0] = np.mean(y==0)
theta_up[0][1] = 1 - theta_up[0][0]

In [92]:
# now update theta_up
for j, sample in enumerate(data):
    # theta_up[0] does not need to be updated
    q_dist = missing_distribution[j]
    for var in range(1,D):
        parent_var = parent_dict[var]
        if var in q_dist and parent_var not in q_dist:
            val_parent = sample[parent_var]
            for val_child in range(2):
                theta_up[var][(val_child, val_parent)] += q_dist[var][val_child]
        elif var not in q_dist and parent_var in q_dist:
            val_child = sample[var]
            for val_parent in range(2):
                theta_up[var][(val_child, val_parent)] += q_dist[parent_var][val_parent]
        elif var in q_dist and parent_var in q_dist:
            for val_child in range(2):
                for val_parent in range(2):
                    theta_up[var][(val_child, val_parent)] += q_dist[var][val_child]*q_dist[parent_var][val_parent]
        else:
            val_child = sample[var]
            val_parent = sample[parent_var]
            theta_up[var][(val_child,val_parent)] += 1

In [94]:
# need to normalize theta_up
for var in range(1,D):
    for val_parent in range(2):
        total = theta_up[var][(0,val_parent)] + theta_up[var][(1,val_parent)]
        theta_up[var][(0,val_parent)] /= total
        theta_up[var][(1,val_parent)] /= total

In [101]:
# update theta using theta_up
theta[0] = theta_up[0][1]
for var in range(1,D):
    for val_parent in range(2):
        theta[var][val_parent] = theta_up[var][(1,val_parent)]

In [118]:
def maximization(data, missing_dist, theta):
    """
    Update the parameters of the model
    """
    # initialize a count dict to count the variables.
    count = {}
    count[0] = {}
    count[0][0] = np.mean(data[:,0]==0)
    count[0][1] = 1 - count[0][0]
    for var in range(1, D):
        count[var] = {}
        for val_parent in range(2):
            count[var][(0,val_parent)] = 0
            count[var][(1,val_parent)] = 0
    # print(count)
    # update the count using the distribution computed in the E-step
    for j, sample in enumerate(data):
        q_dist = missing_distribution[j]
        for var in range(1,D):
            parent_var = parent_dict[var]
            if var in q_dist and parent_var not in q_dist:
                val_parent = sample[parent_var]
                for val_child in range(2):
                    count[var][(val_child, val_parent)] += q_dist[var][val_child]
            elif var not in q_dist and parent_var in q_dist:
                val_child = sample[var]
                for val_parent in range(2):
                    count[var][(val_child, val_parent)] += q_dist[parent_var][val_parent]
            elif var in q_dist and parent_var in q_dist:
                for val_child in range(2):
                    for val_parent in range(2):
                        count[var][(val_child, val_parent)] += q_dist[var][val_child]*q_dist[parent_var][val_parent]
            else:
                val_child = sample[var]
                val_parent = sample[parent_var]
                count[var][(val_child,val_parent)] += 1
    # normalize the count to get probability distributions
    for var in range(1,D):
        for val_parent in range(2):
            total = count[var][(0,val_parent)] + count[var][(1,val_parent)]
            count[var][(0,val_parent)] /= total
            count[var][(1,val_parent)] /= total 
    
    # update theta using theta_up
    theta[0] = count[0][1]
    for var in range(1,D):
        for val_parent in range(2):
            theta[var][val_parent] = count[var][(1,val_parent)]
    return theta


In [119]:
theta = maximization(data, missing_distribution, theta)

In [120]:
theta

{0: 0.38620689655172413,
 1: {0: 0.6008708208274086, 1: 0.24274581072001897},
 2: {0: 0.43229096340931483, 1: 0.599125209210905},
 3: {0: 0.21071323860190072, 1: 0.8902434732585263},
 4: {0: 0.06894679113226365, 1: 0.9816817530172026},
 5: {0: 0.18673563871823387, 1: 0.9278888812895361},
 6: {0: 0.35878762062694197, 1: 0.9357156057172461},
 7: {0: 0.15303882113774459, 1: 0.884255133066244},
 8: {0: 0.9693096794910533, 1: 0.15489908524380214},
 9: {0: 0.9019803128492735, 1: 0.10541680712722212},
 10: {0: 0.6050905404163758, 1: 0.3998007344137381},
 11: {0: 0.5198518500389411, 1: 0.1628680286896265},
 12: {0: 0.16110066487981647, 1: 0.7430563085522784},
 13: {0: 0.19900492161070144, 1: 0.8262186183015763},
 14: {0: 0.32259529008755766, 1: 0.9638602538165654},
 15: {0: 0.6906512504912588, 1: 0.17835389452275507},
 16: {0: 0.6605294542366618, 1: 0.9638268868061921}}

### Test run EM algorithm

In [128]:
D = data.shape[1] # number of features
N = data.shape[0] # number of samples
MISS = -99
parents = {0}
# missing probabilities
b = np.zeros(data.shape[1])
for i in range(D):
    b[i] = np.sum(data[:,i]==MISS) / N 
# parent and child dicts
parent_dict = {0: None, 4: 0, 11: 0, 14: 4, 5: 4, 15: 5, 6: 5, 8: 5, 12: 5,
            9: 5, 13: 5, 7: 8, 3: 8, 1: 12, 2: 13, 16: 7, 10: 2}
child_dict = {0: {4,11}, 4: {5,14}, 5: {15,6,8,12,9,13}, 8: {7,3}, 7: {16},
            13: {2}, 2: {10}, 12: {1}}
# initialize model probabilities
np.random.seed(0)
theta = {}
theta[0] = np.random.random()
for var in range(1,D):
    theta[var] = {}
    for val_parent in range(2):
        theta[var][val_parent] = np.random.random()


In [129]:
num_iters = 10
for i in range(num_iters):
    print(i)
    # E-step
    missing_distribution = []
    for i in range(N):
        missing_distribution.append(gen_missing_var_dist(data[i], theta))
    # M-step
    theta = maximization(data, missing_distribution, theta)
    print(theta[1])
    

0
{0: 0.6136767186533647, 1: 0.22538625550099786}
1
{0: 0.62575930644073618, 1: 0.19439469828462452}
2
{0: 0.62717746644541383, 1: 0.19273821864090263}
3
{0: 0.6272831967008351, 1: 0.1926539913254085}
4
{0: 0.62729012048525112, 1: 0.19264961868762775}
5
{0: 0.62729054043664045, 1: 0.19264943716917934}
6
{0: 0.62729056274592365, 1: 0.19264944617613833}
7
{0: 0.62729056325408483, 1: 0.19264945215265739}
8
{0: 0.62729056306584918, 1: 0.19264945399509886}
9
{0: 0.6272905629961244, 1: 0.19264945450917292}


In [127]:
theta

{0: 0.38620689655172413,
 1: {0: 0.6272905629961244, 1: 0.19264945450917292},
 2: {0: 0.37825460974094732, 1: 0.62358528994857221},
 3: {0: 0.19631725868918934, 1: 0.89981069117926471},
 4: {0: 0.056345201033663084, 1: 0.98250028763911545},
 5: {0: 0.18453846128275811, 1: 0.95490094821711968},
 6: {0: 0.33305796916773489, 1: 0.93993323113544458},
 7: {0: 0.14248205342185821, 1: 0.88733010572916704},
 8: {0: 0.99040569553144631, 1: 0.14752160830839706},
 9: {0: 0.91605795430241743, 1: 0.099531286619424744},
 10: {0: 0.56800381002659139, 1: 0.44318048393889514},
 11: {0: 0.50588235294118644, 1: 0.13207547169817735},
 12: {0: 0.097806799890551896, 1: 0.74400409731855699},
 13: {0: 0.17189576896238146, 1: 0.84736885294283026},
 14: {0: 0.31752043023283621, 1: 0.98196814210418015},
 15: {0: 0.71984749771831302, 1: 0.13633598982683554},
 16: {0: 0.61352628241891205, 1: 0.99320516122383584}}

In [130]:
# initialize model probabilities
np.random.seed(100)
theta = {}
theta[0] = np.random.random()
for var in range(1,D):
    theta[var] = {}
    for val_parent in range(2):
        theta[var][val_parent] = np.random.random()
theta

{0: 0.5434049417909654,
 1: {0: 0.27836938509379616, 1: 0.4245175907491331},
 2: {0: 0.8447761323199037, 1: 0.004718856190972565},
 3: {0: 0.12156912078311422, 1: 0.6707490847267786},
 4: {0: 0.8258527551050476, 1: 0.13670658968495297},
 5: {0: 0.57509332942725, 1: 0.891321954312264},
 6: {0: 0.20920212211718958, 1: 0.18532821955007506},
 7: {0: 0.10837689046425514, 1: 0.21969749262499216},
 8: {0: 0.9786237847073697, 1: 0.8116831490893233},
 9: {0: 0.1719410127325942, 1: 0.8162247487258399},
 10: {0: 0.2740737470416992, 1: 0.4317041836631217},
 11: {0: 0.9400298196223746, 1: 0.8176493787767274},
 12: {0: 0.3361119501208987, 1: 0.17541045374233666},
 13: {0: 0.37283204628992317, 1: 0.005688507352573424},
 14: {0: 0.25242635344484043, 1: 0.7956625084732873},
 15: {0: 0.01525497124633901, 1: 0.5988433769284929},
 16: {0: 0.6038045390428536, 1: 0.10514768541205632}}

In [131]:
num_iters = 10
for i in range(num_iters):
    print(i)
    # E-step
    missing_distribution = []
    for i in range(N):
        missing_distribution.append(gen_missing_var_dist(data[i], theta))
    # M-step
    theta = maximization(data, missing_distribution, theta)
    print(theta[1])
theta

0
{0: 0.5945078561526554, 1: 0.21173325379824082}
1
{0: 0.62485894237331308, 1: 0.19354967527335856}
2
{0: 0.62713470487756251, 1: 0.19268360480475047}
3
{0: 0.62728111441909884, 1: 0.1926500313194765}
4
{0: 0.62729002111305121, 1: 0.19264928135624998}
5
{0: 0.62729053697540971, 1: 0.19264939851915869}
6
{0: 0.62729056305879316, 1: 0.19264943958511729}
7
{0: 0.62729056341471523, 1: 0.19264945065330263}
8
{0: 0.62729056311532394, 1: 0.19264945360914146}
9
{0: 0.62729056301018415, 1: 0.19264945440575335}


{0: 0.38620689655172413,
 1: {0: 0.62729056301018415, 1: 0.19264945440575335},
 2: {0: 0.37825461267532012, 1: 0.62358528940837166},
 3: {0: 0.19631725428914107, 1: 0.89981069518436452},
 4: {0: 0.056345201247827589, 1: 0.98250028810645218},
 5: {0: 0.18453846128489093, 1: 0.95490094850421414},
 6: {0: 0.33305796918195751, 1: 0.93993323129918027},
 7: {0: 0.14248204864717559, 1: 0.88733010321545525},
 8: {0: 0.99040569560097413, 1: 0.14752160279823301},
 9: {0: 0.91605795477387397, 1: 0.099531286364177449},
 10: {0: 0.56800381120723176, 1: 0.44318048308354313},
 11: {0: 0.50588235294119088, 1: 0.13207547169824668},
 12: {0: 0.09780679978782103, 1: 0.74400409769585185},
 13: {0: 0.17189576869811832, 1: 0.8473688530212472},
 14: {0: 0.31752043019976628, 1: 0.98196814211735339},
 15: {0: 0.7198474977007252, 1: 0.13633598955550796},
 16: {0: 0.61352629929805669, 1: 0.99320443504955158}}

In [ ]:
{0: 0.38620689655172413,
 1: {0: 0.6272905629961244, 1: 0.19264945450917292},
 2: {0: 0.37825460974094732, 1: 0.62358528994857221},
 3: {0: 0.19631725868918934, 1: 0.89981069117926471},
 4: {0: 0.056345201033663084, 1: 0.98250028763911545},
 5: {0: 0.18453846128275811, 1: 0.95490094821711968},
 6: {0: 0.33305796916773489, 1: 0.93993323113544458},
 7: {0: 0.14248205342185821, 1: 0.88733010572916704},
 8: {0: 0.99040569553144631, 1: 0.14752160830839706},
 9: {0: 0.91605795430241743, 1: 0.099531286619424744},
 10: {0: 0.56800381002659139, 1: 0.44318048393889514},
 11: {0: 0.50588235294118644, 1: 0.13207547169817735},
 12: {0: 0.097806799890551896, 1: 0.74400409731855699},
 13: {0: 0.17189576896238146, 1: 0.84736885294283026},
 14: {0: 0.31752043023283621, 1: 0.98196814210418015},
 15: {0: 0.71984749771831302, 1: 0.13633598982683554},
 16: {0: 0.61352628241891205, 1: 0.99320516122383584}}

In [132]:
# initialize model probabilities
np.random.seed(300)
theta = {}
theta[0] = np.random.random()
for var in range(1,D):
    theta[var] = {}
    for val_parent in range(2):
        theta[var][val_parent] = np.random.random()
        
num_iters = 10
for i in range(num_iters):
    print(i)
    # E-step
    missing_distribution = []
    for i in range(N):
        missing_distribution.append(gen_missing_var_dist(data[i], theta))
    # M-step
    theta = maximization(data, missing_distribution, theta)
    print(theta[1])
theta

0
{0: 0.5970845505267366, 1: 0.22445937034599997}
1
{0: 0.62503891390647726, 1: 0.19470753535421428}
2
{0: 0.627139868185278, 1: 0.1927764940938631}
3
{0: 0.62728091392814633, 1: 0.19265761980343546}
4
{0: 0.62728996087539257, 1: 0.19264997329473518}
5
{0: 0.62729052684549347, 1: 0.19264947973639768}
6
{0: 0.62729056115149706, 1: 0.19264945320236446}
7
{0: 0.62729056298214125, 1: 0.19264945366734051}
8
{0: 0.62729056300579789, 1: 0.19264945436996966}
9
{0: 0.62729056298119068, 1: 0.19264945460719479}


{0: 0.38620689655172413,
 1: {0: 0.62729056298119068, 1: 0.19264945460719479},
 2: {0: 0.37825461007786793, 1: 0.62358528961547643},
 3: {0: 0.19631726292740803, 1: 0.89981068731425884},
 4: {0: 0.056345200827605503, 1: 0.98250028718875715},
 5: {0: 0.18453846128691842, 1: 0.9549009479490117},
 6: {0: 0.33305796915007135, 1: 0.93993323097969184},
 7: {0: 0.14248205868900932, 1: 0.88733010825245273},
 8: {0: 0.99040569546550605, 1: 0.1475216135889322},
 9: {0: 0.91605795385235456, 1: 0.099531286866340801},
 10: {0: 0.56800381020026813, 1: 0.44318048384040926},
 11: {0: 0.50588235294117367, 1: 0.13207547169816536},
 12: {0: 0.097806799990425825, 1: 0.74400409695069514},
 13: {0: 0.1718957689376652, 1: 0.84736885265515816},
 14: {0: 0.31752043026295584, 1: 0.98196814210266481},
 15: {0: 0.71984749773905954, 1: 0.13633599008792854},
 16: {0: 0.61352621956915798, 1: 0.99320581987332257}}